In [2]:
# Setup the google colab
# !pip install xmnlp

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
isColab = False
googlePath = "./drive/MyDrive/L101Project/" if isColab else "./"
cpuPools = 2 if isColab else 8

In [2]:
import tensorflow as tf
from tensorflow import keras
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [3]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import Session

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = Session(config=config)
# tf.compat.v1.disable_eager_execution()


In [4]:
import csv
import numpy as np
import xmnlp
import itertools
from collections import Counter
from keras.utils import to_categorical

import collections
import operator
from functools import reduce
import json
from multiprocessing import Pool
from keras.preprocessing.sequence import pad_sequences

In [5]:
# Preprocess the training and testing data.

In [6]:
pkuTrain = googlePath + "./corpus/cws/icwb2-data/training/pku_training.utf8"
pkuTest = googlePath + "./corpus/cws/icwb2-data/testing/pku_test.utf8"

f = open(pkuTest, "r")
temp = f.readlines()
con = 0
for item in temp:
    con+=1
    if(item == "\n"):
        print(con)

In [7]:
## Create Corpus for Chinese Word Segamentation

In [8]:
import pandas as pd
pkuTrain = googlePath + "./corpus/cws/icwb2-data/training/pku_training.utf8"
pkuTest = googlePath + "./corpus/cws/icwb2-data/testing/pku_test.utf8"
train = pd.read_table(pkuTrain,  encoding='utf8', header=None, names=['input'])  # don't drop the empty lines yet, they show up as NaN in the data frame
# train.head(n=1)


In [9]:
pkuTest = googlePath  + "./corpus/cws/icwb2-data/testing/pku_test.utf8"
test = pd.read_table(pkuTest,  encoding='utf8', header=None, names=['input']) 
test["raws"] = test["input"]
# test.head(n=1)

In [10]:
def prepross_data(train_Data):
    inputs = train_Data["input"][:]
    print(len(inputs))
    raws = []
    tokenList = []
    
    for item in inputs:
        item = item.replace("   ", "  ")

        raw = item.replace(" ", "")
#         print(item)
#         print(raw)
        raws.append(raw)
        tokens = (item.split("  ")[:-1])
        tokenList.append(tokens)
#         print(tokens)
    train_Data["raws"] = raws
    train_Data["tokenList"] = tokenList
    dictionary_train_word = list(set(reduce(operator.add,train_Data["tokenList"])))
    dictionary_train_char =  list(set(reduce(operator.add,train_Data["raws"])))    
    return train_Data, dictionary_train_char, dictionary_train_word
train, train_dic_char, train_dic_word  = prepross_data(train)

19054


In [11]:
# Get redical dictionary

# Get redical of a chinese character:
def str_get_redical(st):
    return xmnlp.radical(st)

def get_redical_dic_from_char_dic(char_dic):
    return list(set(str_get_redical("".join(char_dic))))

train_dic_redical = get_redical_dic_from_char_dic(train_dic_char)
print(len(train_dic_redical))

(Lazy Load) Loading model...
235


In [12]:
# Build up subcharacter dictionary
# Build up Chinese sub-character unites diactionary.
chaiZi_Dic = {}
with open(googlePath + "corpus/chaizi/chaizi-jt.txt", 'r') as f:
    reader = csv.reader(f,delimiter='\t')
    for row in reader:
        chaiZi_Dic[row[0]] = row[1]
        
def getSubChar(charCN):
    if(charCN in chaiZi_Dic.keys()):
        return chaiZi_Dic[charCN].split(" ")
#     elif charCN in train_dic_char:
#         return [charCN]
    else:
        return [None]

In [13]:
def str_get_subchar(st):
#     for item in st:
#         print(item)
#         print(getSubChar(item))
    return [getSubChar(item) for item in st]

def get_subchar_dic_from_char_dic(char_dic):
    return list(set(itertools.chain.from_iterable(str_get_subchar("".join(char_dic)))))


def get_top200_subchar_dic_from_char_dic(char_dic):
    allsub = (list(itertools.chain.from_iterable(str_get_subchar("".join(char_dic)))))
    c = Counter(allsub)
    ans = list(map(lambda x : x[0] , (c.most_common(200))))
    return ans
# train_dic_subchar = get_subchar_dic_from_char_dic(train_dic_char)
train_dic_subchar_top200 = get_top200_subchar_dic_from_char_dic(train_dic_char)


train_dic_subchar = get_subchar_dic_from_char_dic(train_dic_char)
print(len(train_dic_subchar))

1191


In [14]:
# Validate the pre-processing
def validate_data(data):
    tkList = data["tokenList"]
    raws = data["raws"]
    for (tokens, raw) in zip(tkList,raws):
#         print(tokens)
        
#         print(raw)
        temp  = "".join(tokens)
#         print(temp)
        assert(temp == raw)
        if not (temp == raw):
            print(temp)
            print(raw)
            print(tokens)
validate_data(train)

In [15]:
# print(len(train_dic_char))
# print(len(set(train_dic_char)))
# print(len(train_dic_word))
# print(len(set(train_dic_word)))
# with open(googlePath + "./corpus/cws/icwb2-data/gold/pku_training_words.utf8", 'r') as f:
#     content = f.readlines()  
#     print(len(content))

# Make bag of subchar model.


# 	letter = 
# 	letter[value] = 1



# str_to_bagOfChar("迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）")[2][1]

In [16]:
# Feature extraction

oov = len(train_dic_char)
print(oov)

oov_redical = len(train_dic_redical)
print(oov_redical)

oov_subchar = len(train_dic_subchar)
print(oov_subchar)

def token_index(tok):
    ind = tok
    if tok in train_dic_char:  # if token in vocabulary
        ind = train_dic_char.index(tok)
    else:  # else it's OOV
        ind = oov
    return ind

def str_token_index(string):
    return [token_index(x) for x in (string)]




def redical_index(red):
    ind = oov_redical
    if red in train_dic_redical:  # if token in vocabulary
        ind = train_dic_redical.index(red)
    return ind

def str_red_index(string):
    return [redical_index(x) for x in str_get_redical(string)]


def subchar_index(subchar):
    ind = oov_subchar
    if subchar in train_dic_subchar:
        ind = train_dic_subchar.index(subchar)
    return ind

def str_subchar_index(string):
    return [[subchar_index(temp) for temp in x] for x in str_get_subchar(string)]

def char_to_bagOfChar(cnChar):
    ans = [0 for _ in range(len(train_dic_subchar_top200)+1)]
    for item in (getSubChar(cnChar)):
        if item in train_dic_subchar_top200:
            idx = train_dic_subchar_top200.index(item)
        else:
            idx = len(train_dic_subchar_top200)
        ans[idx] += 1
    return (ans)

def str_to_bagOfChar(cnStr):
    return [char_to_bagOfChar(cnChar) for cnChar in cnStr]
        



# Get Begin Middle End Single sequence
# B 0 M 1 E 2 S 3

def str_bmes_idx(tokenList):
    answer = []
    for item in tokenList:
        if len(item) == 0:
            raise NameErro("Zero Length Word")
        if len(item) == 1:
            answer.append(3)
        else:
            answer.append(0)
            for item in range(len(item) - 2):
                answer.append(1)
            answer.append(2)
    return answer

def extract_features(data_set, isTest=False):
    data_temp = data_set.copy()

    # Idx for chars
    with Pool(8) as p:
        tokinds = p.map(str_token_index,data_temp['raws'])
#     tokinds = [list(map(token_index, u)) for u in data_temp['raws']]

    with Pool(8) as p:
        redinds = p.map(str_red_index,data_temp['raws'])
        
    with Pool(8) as p:
        subcharidx = p.map(str_subchar_index,data_temp['raws'])
        
#     subcharidx_bag = list(m0ap(str_to_bagOfChar,data_temp['raws']))
        

    data_temp["tokenIdx"] = tokinds
    data_temp["redIdx"] = redinds
    data_temp["subcharIdx"] = subcharidx
#     data_temp["subcharIdxBag"] = subcharidx_bag
    
    # BIO
    if(not isTest):
        data_temp["bmes"] = [str_bmes_idx(u) for u in data_temp['tokenList']]
        assert (list(map(len,data_temp["bmes"])) == list(map(len,data_temp["tokenIdx"])))
    
    return data_temp

4698
235
1191


In [17]:
%%time
# MultiThreading
train_feature = extract_features(train)
# train_feature.head(5)

CPU times: user 2.51 s, sys: 304 ms, total: 2.82 s
Wall time: 24.6 s


In [18]:
train_feature.head(1)

,input,raws,tokenList,tokenIdx,redIdx,subcharIdx,bmes
0,迈向 充满 希望 的 新 世纪 —— 一九九八年 新年 讲话 （ 附 ...,迈向充满希望的新世纪——一九九八年新年讲话（附图片１张）,"[迈向, 充满, 希望, 的, 新, 世纪, ——, 一九九八年, 新年, 讲话, （, 附...","[2820, 2896, 2538, 1719, 4222, 983, 3642, 3029...","[168, 64, 34, 26, 192, 228, 73, 36, 6, 28, 195...","[[608, 393], [154, 22, 1001], [1188, 157], [83...","[0, 2, 0, 2, 0, 2, 3, 3, 0, 2, 0, 2, 0, 1, 1, ..."


In [19]:
def find_longest_sequence(data_with_features):
#     assert (np.max(list(map(len, data_with_features["tokenIdx"])))) == (np.max(list(map(len, data_with_features["bmes"]))))
    return (np.max(list(map(len, data_with_features["tokenIdx"]))))
    
    

In [20]:
%%time
temp = extract_features(test, isTest=True)
test_feature = temp
# test_feature.head(5)

CPU times: user 153 ms, sys: 220 ms, total: 373 ms
Wall time: 2.84 s


In [21]:
test_feature.head(5)

,input,raws,tokenIdx,redIdx,subcharIdx
0,共同创造美好的新世纪——二○○一年新年贺词,共同创造美好的新世纪——二○○一年新年贺词,"[592, 2632, 3405, 2988, 4161, 1081, 3642, 3029...","[125, 64, 33, 168, 196, 214, 73, 36, 6, 28, 19...","[[771, 164], [22, 923, 1001], [636, 282], [608..."
1,（二○○○年十二月三十一日）（附图片1张）,（二○○○年十二月三十一日）（附图片1张）,"[4390, 2666, 735, 735, 735, 1740, 1585, 2666, ...","[195, 24, 195, 195, 195, 146, 81, 24, 228, 6, ...","[[262], [923, 923], [262], [262], [262], [537,..."
2,女士们，先生们，同志们，朋友们：,女士们，先生们，同志们，朋友们：,"[1438, 1228, 1236, 2207, 1969, 1454, 1236, 220...","[214, 42, 92, 195, 34, 229, 92, 195, 64, 153, ...","[[561, 923, 537], [440, 923], [306, 142], [262..."
3,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...,2001年新年钟声即将敲响。人类社会前进的航船就要驶入21世纪的新航程。中国人民进入了向现代...,"[4191, 546, 546, 2763, 1740, 3029, 1740, 2546,...","[195, 195, 195, 195, 146, 36, 146, 84, 42, 225...","[[262], [262], [262], [262], [537, 255, 154], ..."
4,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...,在这个激动人心的时刻，我很高兴通过中国国际广播电台、中央人民广播电台和中央电视台，向全国各族...,"[2203, 3380, 167, 3536, 2848, 3787, 2527, 3642...","[1, 168, 162, 26, 32, 231, 153, 73, 111, 33, 1...","[[923, 306, 913], [608, 325], [306, 813], [830..."


In [22]:
train_longest = find_longest_sequence(train_feature)
print(train_longest)
test_longest = find_longest_sequence(test_feature)
print(test_longest)
seq_longest = np.max([train_longest,test_longest])
print(seq_longest)

subchar_seq_length = np.max(list(map(lambda x : len(getSubChar(x)),train_dic_char)))
subchar_seq_length = 3
subsubchar_padtok = oov_subchar + 1
seq_length = seq_longest

def padd_char(seq):
    temp_char_seqs_padded = []
    for item in seq["subcharIdx"]:
        temp_pad = pad_sequences(item, maxlen=subchar_seq_length,
                                  dtype='int32', padding='post', truncating='post', value=subsubchar_padtok)
  
        a = temp_pad
        b = [[subsubchar_padtok for i in range(subchar_seq_length)] for _ in range(0, seq_length - len(temp_pad))]
        if len(b) == 0:
            c = a
        else:
            c = np.concatenate((a, b))
        
    # print(len(c))
        temp_char_seqs_padded.append(c)
  # print(len(temp_char_seqs_padded))
    return temp_char_seqs_padded

1019
626
1019


In [23]:
train_subchar_seqs_padded = padd_char(train_feature)


In [24]:
seq_length = seq_longest

# a new dummy token index, one more than OOV
padtok = oov+1
red_padtok = oov_redical + 1
print('The padding token index is %i' % padtok)

# use pad_sequences, padding or truncating at the end of the sequence (default is 'pre')
train_seqs_padded = pad_sequences(train_feature['tokenIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padtok)
print('Example of padded token sequence:')
print(train_seqs_padded[1])


train_red_padded = pad_sequences(train_feature['redIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=red_padtok)


train_subchar_seqs_padded = padd_char(train_feature)





# Prepare Test set.
test_seqs_padded = pad_sequences(test_feature['tokenIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padtok)

test_rad_padded = pad_sequences(test_feature['redIdx'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=red_padtok)

test_subchar_seqs_padded = padd_char(test_feature)


The padding token index is 4699
Example of padded token sequence:
[3548  592 3548 ... 4699 4699 4699]


In [25]:
# get lists of named entity labels, padded with a null label (=3)
padlab = 4
train_labs_padded = pad_sequences(train_feature['bmes'].tolist(), maxlen=seq_length,
                                  dtype='int32', padding='post', truncating='post', value=padlab)

# convert those labels to one-hot encoding
n_labs = 5
train_labs_onehot = [to_categorical(i, num_classes=n_labs) for i in train_labs_padded]

# # follow the print outputs below to see how the labels are transformed
# print('Length of input sequence: %i' % len(train_labs_padded[1]))
# print('Length of label sequence: %i' % len(train_labs_onehot[1]))
# print(train_labs_padded[1][:11])
# print(train_labs_onehot[1][:11])

In [33]:
# load Keras and TensorFlow



# our final vocab size is the padding token + 1 (OR length of vocab + OOV + PAD)
vocab_size = padtok+1
red_size = red_padtok+1
subchar_size = subsubchar_padtok + 1

print(vocab_size==len(train_dic_char)+2)
embed_size = 128 # y an embedding size of 128 (could tune this)

# list of metrics to use: true & false positives, negatives, accuracy, precision, recall, area under the curve
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# our model has the option for an label prediction bias, it's sequential, starts with an embedding layer, then bi-LSTM,
# a dropout layer follows for regularisation, and a dense final layer with softmax activation to output class probabilities
# we compile with the Adam optimizer at a low learning rate, use categorical cross-entropy as our loss function
def make_model(metrics = METRICS, output_bias=None):
    if output_bias is not None:        
        output_bias = tf.keras.initializers.Constant(output_bias)
    tok_input1 = keras.layers.Input(shape=(seq_length,), dtype='int32', name='tok_input1')
    red_input2 = keras.layers.Input(shape=(seq_length,), dtype='int32', name='red_input2')
    subchar_input3 = keras.layers.Input(shape=(seq_length,subchar_seq_length), dtype='int32', name='char_input3')

    emb_char = keras.layers.TimeDistributed(keras.layers.Embedding(output_dim=embed_size, input_dim=subchar_size, input_length=3,  mask_zero=True, trainable=True))(subchar_input3)
#     emb_char = keras.layers.TimeDistributed(keras.layers.Embedding(output_dim=embed_size, input_dim=subchar_size, input_length=3,  mask_zero=True, trainable=True))(subchar_input3)
    
    char_enc = keras.layers.TimeDistributed(keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=False, dropout=0.2,recurrent_dropout=0.2)))(emb_char)
        

    x1 = keras.layers.Embedding(output_dim=embed_size, input_dim=vocab_size,  input_length=seq_length,  mask_zero=True, trainable=True)(tok_input1)
    x2 = keras.layers.Embedding(output_dim=embed_size, input_dim=red_size,  input_length=seq_length, mask_zero=True, trainable=True)(red_input2)
    x_cancat = keras.layers.concatenate([x1, x2, char_enc])
#     x_cancat = (x1)
    x_lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=True, dropout=0.2))(x_cancat)
    x_drop = keras.layers.Dropout(0.5)(x_lstm)
    main_output = keras.layers.TimeDistributed(keras.layers.Dense(n_labs, activation='softmax', bias_initializer=output_bias))(x_drop)
    model = keras.models.Model(inputs=[tok_input1,red_input2, subchar_input3], outputs= main_output)
#     model = keras.models.Model(inputs=[tok_input1, pos_input2, char_input3], outputs= main_output)
    
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-3), loss=keras.losses.CategoricalCrossentropy(), metrics=metrics)
    return model


# Old lstm model
#     model = keras.Sequential([
#         keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=seq_length, mask_zero=True, trainable=True),
#         keras.layers.Bidirectional(keras.layers.LSTM(units=50, return_sequences=True, dropout=0.2)),  # 2 directions, 50 units each, concatenated (can change this)
#         keras.layers.Dropout(0.5),
#         keras.layers.TimeDistributed(keras.layers.Dense(n_labs, activation='softmax', bias_initializer=output_bias)),
#     ])


# early stopping criteria based on area under the curve: will stop if no improvement after 10 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', verbose=1, patience=10, mode='max', restore_best_weights=True)

# the number of training epochs we'll use, and the batch size (how many texts are input at once)
EPOCHS = 100
BATCH_SIZE = 128

print('**Defining a neural network**')
model = make_model()
model.summary()

True
**Defining a neural network**
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input3 (InputLayer)        [(None, 1019, 3)]    0                                            
__________________________________________________________________________________________________
tok_input1 (InputLayer)         [(None, 1019)]       0                                            
__________________________________________________________________________________________________
red_input2 (InputLayer)         [(None, 1019)]       0                                            
__________________________________________________________________________________________________
time_distributed_8 (TimeDistrib (None, 1019, 3, 128) 152704      char_input3[0][0]                
_________________________________________________________

In [27]:
# %%time
# # evaluate our initial model
# results = model.evaluate(X, y, batch_size=BATCH_SIZE, verbose=0)
# print("Loss: {:0.4f}".format(results[0]))

In [31]:
# figure out the label distribution in our fixed-length texts
from collections import Counter
all_labs = [l for lab in train_labs_padded for l in lab]
label_count = Counter(all_labs)
total_labs = len(all_labs)
print(label_count)
print(total_labs)

# use this to define an initial model bias
initial_bias=[(label_count[0]/total_labs), (label_count[1]/total_labs),
              (label_count[2]/total_labs), (label_count[3]/total_labs), (label_count[4]/total_labs)]
print('Initial bias:')
print(initial_bias)

Counter({4: 17589578, 0: 585226, 2: 585226, 3: 524721, 1: 131275})
19416026
Initial bias:
[0.03014138938627297, 0.006761167295511451, 0.03014138938627297, 0.027025149224666263, 0.9059309047072763]


In [32]:
# prepare sequences and labels as numpy arrays, check dimensions
X = np.array(train_seqs_padded)
X_red = np.array(train_red_padded)
X_sub = np.array(train_subchar_seqs_padded)
y = np.array(train_labs_onehot)
print('Input sequence dimensions (n.docs, seq.length):')
print(X.shape)
print('Label dimensions (n.docs, seq.length, one-hot encoding of 4 NER labels):')
print(y.shape)

Input sequence dimensions (n.docs, seq.length):
(19054, 1019)
Label dimensions (n.docs, seq.length, one-hot encoding of 4 NER labels):
(19054, 1019, 5)


In [33]:
# re-initiate model with bias
model = make_model(output_bias=initial_bias)
# and fit...
model.fit([X,X_red,X_sub], y, batch_size=128, epochs=10, callbacks = [early_stopping],  validation_split = 0.3)

Epoch 1/10
13337/13337 [==============================] - 994s 75ms/step - loss: 0.0376 - tp: 13368166.0000 - fp: 134333.0000 - tn: 54227164.0000 - fn: 222237.0000 - accuracy: 0.9948 - precision: 0.9901 - recall: 0.9836 - auc: 0.9997 - val_loss: 0.0258 - val_tp: 5766751.0000 - val_fp: 44656.0000 - val_tn: 23257868.0000 - val_fn: 58872.0000 - val_accuracy: 0.9964 - val_precision: 0.9923 - val_recall: 0.9899 - val_auc: 0.9998
Epoch 2/10
13337/13337 [==============================] - 1009s 76ms/step - loss: 0.0246 - tp: 13460860.0000 - fp: 97751.0000 - tn: 54263776.0000 - fn: 129543.0000 - accuracy: 0.9967 - precision: 0.9928 - recall: 0.9905 - auc: 0.9998 - val_loss: 0.0221 - val_tp: 5775202.0000 - val_fp: 38601.0000 - val_tn: 23263868.0000 - val_fn: 50421.0000 - val_accuracy: 0.9969 - val_precision: 0.9934 - val_recall: 0.9913 - val_auc: 0.9998
Epoch 3/10
13337/13337 [==============================] - 1080s 81ms/step - loss: 0.0215 - tp: 13478432.0000 - fp: 86595.0000 - tn: 54275208.000

In [34]:
X_test = np.array(test_seqs_padded)
X_test_rad = np.array(test_rad_padded)
X_test_subchar = np.array(test_subchar_seqs_padded)

# preds = np.argmax(model.predict(X_test), axis=-1)
preds = np.argmax(model.predict([X_test,X_test_rad,X_test_subchar]), axis=-1)
flat_preds = [p for pred in preds for p in pred]
print(Counter(flat_preds))

Counter({4: 1808203, 2: 57461, 0: 57383, 3: 45354, 1: 12535})


In [35]:
len(preds)

1944

In [36]:
def postEditPred(pred):
    print(pred)
    ans = []
    for item in pred:
        if item == 0 or item == 3:
            ans.append(1)
        else:
            ans.append(0)
            
    assert len(ans) == len(pred)
    return ans

def splitSentence(st, pred):
    temp_st = st
    temp_pred = pred
    temp_pred[0] = 2
    buf = []
    result = []
    for (pre,char) in zip(temp_pred,temp_st):
        if(pre == 0 or pre == 3):
            result.append(buf)
            buf = []
        buf.append(char)
    if(len(buf) > 0):
        result.append(buf)
    return result
    
    



def sentencePrediction(dataset,prediction):
    data_temp = dataset.copy()
    assert len(data_temp) == len(prediction)
    raw_sents = data_temp["raws"]
    assert len(raw_sents) == len(prediction)
    ans = []
    for (st,pred) in zip(raw_sents,prediction):
        sptSt = splitSentence(st,pred)
        ans.append(sptSt)
    assert len(ans) == len(data_temp)
    data_temp["tokenList"]= ans
    return data_temp
    
def convertToPredSts(dataset):
    data_temp = dataset.copy()
    tokenList = data_temp["tokenList"]
    ans = []
    
    for item in tokenList:
        temp_st = list(map(lambda x : "".join(x), item))
        temp = "  ".join(temp_st)
        temp = temp + "  "
        ans.append(temp)
    return ans

In [37]:
result_pred = sentencePrediction(test_feature,preds)
output_sts = convertToPredSts(result_pred)

f = open("./ans.txt", "w")
f.write("\n".join(output_sts))
f.close()

In [ ]:
f = open(googlePath + "./this.txt", "w")
f.write("\n".join(output_sts))
f.close()